In [14]:
import sys
sys.path.append("V:/biomoni/") 
from biomoni import Experiment
from BacillusScripts.BacillusVariableFeedrate_copy import Bacillus_vf
from biomoni import Model
import scipy
from scipy.interpolate import interp1d
import pandas as pd
import numpy as np

In [ ]:
path = r"V:/biomoni/BacillusData/Stamm185"

experiment_dict_for_estimation = {exp : Experiment(path, exp,endpoint = "Sim_end") for exp in ["F5"]}  #all experiments in a dictionary   
experiment_dict_for_graphs = {exp : Experiment(path, exp, endpoint = "F_end") for exp in ["F5"]} 



#Exp = experiment_dict 
b= Bacillus_vf()
b.estimate(experiment_dict_for_estimation)
b.report()
print(b.p)


In [17]:
t_grid = np.linspace(0,64,1001) 
sim_dict_all = {experiment.exp_id: b.simulate(experiment = experiment, t_grid = t_grid) for experiment in experiment_dict_for_estimation.values()} 

CRR_all ={}

for key in experiment_dict_for_graphs.keys():
    CRR_all[key]= {}

for key, dat in sim_dict_all.items():
    V_func= interp1d(x=sim_dict_all[key]["V"].index, y=sim_dict_all[key]["V"], fill_value = (sim_dict_all[key]["V"].iloc[0], sim_dict_all[key]["V"].iloc[-1]) , bounds_error= False)
    exp =Experiment(path, key, endpoint="F_end")
    c = b.create_controls(exp)
    y0 = b.create_y0(exp)
    #datasaet = exp.dataset
    
    CRR1, CRR2 = b.calc_CRR(experiment=exp,c=c,V=V_func,y0=y0)
    
    CRR_all[key]= {"CR1": CRR1,"CR2": CRR2}
    
    
print(CRR_all)
    




mX0 changed to 0.0133800816825


V:\biomoni\BacillusScripts\BacillusVariableFeedrate_copy.py:796: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  cummulation = scipy.integrate.quad(func,0,t,limit=200)
V:\biomoni\BacillusScripts\BacillusVariableFeedrate_copy.py:796: IntegrationWarning: The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  cummulation = scipy.integrate.quad(func,0,t,limit=200)
V:\biomoni\BacillusScripts\BacillusVariableFeedrate_copy.py:822: IntegrationWarning: The maximum number of subdivisions (200) has be

<class 'pandas.core.frame.DataFrame'>
{'F5': {'CR1':            feedrate pure glucose g/h   Glc g_cum     CO2 %   V_CO2 L  \
0.069444                    0.000000    0.000000  0.047000  0.002100   
16.069444                   0.000000    0.000000  0.207583  0.050275   
17.752778                   0.000000    0.000000  0.335067  0.088520   
18.586111                   3.297837    3.024035  0.397650  0.107295   
19.836111                   3.611917    7.351651  0.496550  0.136965   
20.802778                   3.856201   10.964868  0.592650  0.165795   
21.702778                   4.083037   14.536210  0.673450  0.190035   
22.852778                   4.362218   19.393836  0.784500  0.223350   
39.452778                   6.630572  119.197885  4.941500  1.470450   
40.719444                   6.630572  127.597166  4.936000  1.468800   
41.952778                   6.630572  135.775234  4.782500  1.422750   
43.402778                   6.630572  145.483693  4.406500  1.309950   
44.736111  

In [ ]:
a = Experiment(path,"F5")
y0 = b.create_y0(a)
c = b.create_controls(a)
path = r"V:/biomoni/BacillusData/Stamm185"
t_grid = np.linspace(0,40,1001) 
experiment_dict_for_estimation = {exp : Experiment(path, exp,endpoint = "Sim_end") for exp in ["F5"]}  #all experiments in a dictionary   
experiment_dict_for_graphs = {exp : Experiment(path, exp, endpoint = "F_end") for exp in ["F5"]} 

b.estimate(experiment_dict_for_estimation)
b.report()
print(b.p)
t_grid = np.linspace(0,40,1001) 
sim_dict_all = {experiment.exp_id: b.simulate(experiment = experiment, t_grid = t_grid) for experiment in experiment_dict_for_estimation.values()} 

print(sim_dict_all)

In [ ]:

a.dataset["CO2"]["V_CO2 L"] = (a.dataset["CO2"]["CO2"]- 0.04)/100 * 30   
print(a.dataset["CO2"])


CO2_func = interp1d(x = a.dataset["CO2"].index, 
                    y = a.dataset["CO2"]["V_CO2 L"], fill_value = (a.dataset["CO2"]["V_CO2 L"].iloc[0], a.dataset["CO2"]["V_CO2 L"].iloc[-1]) ,
                    bounds_error= False)

In [80]:
t_grid = list(a.dataset["off"].index.values)

Carb_RR = []
V = 0.5
Mw_bm = 24.445          #g/mol molecular weight of biomass assuming composition of C=1,H=1.594,N=0.293,O=0.387,P=0.012,S=0.005
Mw_gluc = 180.156       #g/mol molecular weight of glucose C=6,H=12,O=6
Mw_RF = 376.36          #g/mol molecular weight of Riboflavin C=17,H=20,N=4,O=6
dV_gas_dt = c["gas_flow"]
R = 0.08314 #bar*l/mol*K
T = c["T"] + 273.15     #Kelvin  
pressure = c["pressure"] #bar
Mw_CO2 = 44.01

F_C_in = c["feedrate_glc"](t_grid)
df = pd.DataFrame(data=F_C_in,index=t_grid, columns=["feedrate pure glucose g/h"])

# calculate cumulated amount of Glucose from Feed
F_C_in_cum = []

for t in t_grid:
    func = c["feedrate_glc"]
    cummulation = scipy.integrate.quad(func,0,t,limit=200)
    
    F_C_in_cum.append(cummulation)

df["Glc g_cum"] = [x[0] for x in F_C_in_cum]


#creating column with calculcated Volume values for CO2 [L] from exhaust gas measurment
a.dataset["CO2"]["V_CO2 L"] = (a.dataset["CO2"]["CO2"]- 0.04)/100 * 30   #30 L/h is F_exhaust or Outlet flow (assuming = inlet flow)

CO2_func = interp1d(x = a.dataset["CO2"].index, 
                    y = a.dataset["CO2"]["CO2"], fill_value = (a.dataset["CO2"]["CO2"].iloc[0], a.dataset["CO2"]["CO2"].iloc[-1]) , 
                    bounds_error= False)

V_CO2_func = interp1d(x = a.dataset["CO2"].index, 
                    y = a.dataset["CO2"]["V_CO2 L"], fill_value = (a.dataset["CO2"]["V_CO2 L"].iloc[0], a.dataset["CO2"]["V_CO2 L"].iloc[-1]) ,
                    bounds_error= False)

df["CO2 %"] = CO2_func(t_grid)
df["V_CO2 L"] = V_CO2_func(t_grid)



#calculate cumulated amount of CO2 in exhaust gas 
CO2_out_cum = []

for t in t_grid:
    func = V_CO2_func     
    cummulation = scipy.integrate.quad(func,0,t,limit=200)
    
    CO2_out_cum.append(cummulation)
    
df["CO2 L_cum"] = [x[0] for x in CO2_out_cum]

df["CO2 mol_cum"] = (df["CO2 L_cum"] * pressure) / (R * T)

#print(df)

Carb_RR= {}
Carb_RR["t"]= t_grid
Carb_RR["given"]=[]
Carb_RR["found"]=[]
Carb_RR["RR"]=[]

for t in t_grid:
    
    #mass balance of Carbon given (Substrate,Biomass,Product)
    nC_Sg = y0[1]/Mw_gluc *6 + df["Glc g_cum"].values[t_grid.index(t)]/Mw_gluc * 6   #initial Substrate + Substrate from Feed
    nC_Xg = y0[0]/Mw_bm
    nC_Pg = y0[2]/Mw_RF *17
    
    amountC_given = nC_Sg + nC_Xg + nC_Pg 
    
    
    
    #mass balance of Carbon found (Substrate,Biomass,Product,CO2)
    nC_Sf = (float(a.dataset["off"]["Glucose [g/L]"].loc[[t]])*V)/Mw_gluc * 6
    nC_Xf = (float(a.dataset["off"]["CDW_calc"].loc[[t]])*V)/ Mw_bm
    nC_Pf = (float(a.dataset["off"]["RF [mg/L]"].loc[[t]])*V /1000)/ Mw_RF * 17
    nC_CO2 = float(df["CO2 mol_cum"].values[t_grid.index(t)])/Mw_CO2

    amountC_found = nC_Sf + nC_Xf + nC_Pf + nC_CO2
    
    RR = (amountC_found / amountC_given) * 100
    
    Carb_RR["given"].append(amountC_given)
    Carb_RR["found"].append(amountC_found)
   
    Carb_RR["RR"].append(RR)
 
df2 = pd.DataFrame.from_dict(Carb_RR)
 

df["RR"] = df2["RR"].values 


    
    
#g = a.dataset["off"]["Glucose [g/L]"].loc[[0.0694444]]
#f= a.dataset["off"]["Glucose [g/L]"]
#print(t_grid)
#print(Carb_RR)

#print(f)

C:\Users\walkyre\AppData\Local\Temp/ipykernel_10564/1137407779.py:22: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

C:\Users\walkyre\AppData\Local\Temp/ipykernel_10564/1137407779.py:22: IntegrationWarning:

The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.

C:\Users\walkyre\AppData\Local\Temp/ipykernel_10564/1137407779.py:50: IntegrationWarning:

The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the i

            t     given     found         RR
0    0.069444  0.519105  0.509701  98.188543
1   16.069444  0.519105  0.514439  99.101165
2   17.752778  0.519105  0.497285  95.796751
3   18.586111  0.619818  0.515486  83.167313
4   19.836111  0.763947  0.564973  73.954377
5   20.802778  0.884284  0.605875  68.515939
6   21.702778  1.003225  0.658426  65.630965
7   22.852778  1.165006  0.754225  64.740016
8   39.452778  4.488927  1.145557  25.519619
9   40.719444  4.768660  1.227734  25.745880
10  41.952778  5.041027  1.214177  24.085898
11  43.402778  5.364362  1.214518  22.640486
12  44.736111  5.655646  1.203883  21.286389
13  46.052778  5.946413  1.140351  19.177121
14  47.336111  6.199775  1.153407  18.604009
15  63.719444  8.679813  0.564065   6.498585
           feedrate pure glucose g/h   Glc g_cum     CO2 %   V_CO2 L  \
0.069444                    0.000000    0.000000  0.047000  0.002100   
16.069444                   0.000000    0.000000  0.207583  0.050275   
17.752778          

In [82]:
t_grid = list(a.dataset["off"].index.values)


V = 0.5
Mw_bm = 24.445          #g/mol molecular weight of biomass assuming composition of C=1,H=1.594,N=0.293,O=0.387,P=0.012,S=0.005
Mw_gluc = 180.156       #g/mol molecular weight of glucose C=6,H=12,O=6
Mw_RF = 376.36          #g/mol molecular weight of Riboflavin C=17,H=20,N=4,O=6
dV_gas_dt = c["gas_flow"]
R = 0.08314 #bar*l/mol*K
T = c["T"] + 273.15     #Kelvin  
pressure = c["pressure"] #bar
Mw_CO2 = 44.01

F_C_in = c["feedrate_glc"](t_grid)
df = pd.DataFrame(data=F_C_in,index=t_grid, columns=["feedrate pure glucose g/h"])

# calculate cumulated amount of Glucose from Feed
F_C_in_cum = []

for t in t_grid:
    func = c["feedrate_glc"]
    cummulation = scipy.integrate.quad(func,0,t,limit=200)
    
    F_C_in_cum.append(cummulation)

df["Glc g_cum"] = [x[0] for x in F_C_in_cum]


#creating column with calculcated Volume values for CO2 [L] from exhaust gas measurment
a.dataset["CO2"]["V_CO2 L"] = (a.dataset["CO2"]["CO2"]- 0.04)/100 * 30   #30 L/h is F_exhaust or Outlet flow (assuming = inlet flow)

CO2_func = interp1d(x = a.dataset["CO2"].index, 
                    y = a.dataset["CO2"]["CO2"], fill_value = (a.dataset["CO2"]["CO2"].iloc[0], a.dataset["CO2"]["CO2"].iloc[-1]) , 
                    bounds_error= False)

V_CO2_func = interp1d(x = a.dataset["CO2"].index, 
                    y = a.dataset["CO2"]["V_CO2 L"], fill_value = (a.dataset["CO2"]["V_CO2 L"].iloc[0], a.dataset["CO2"]["V_CO2 L"].iloc[-1]) ,
                    bounds_error= False)

df["CO2 %"] = CO2_func(t_grid)
df["V_CO2 L"] = V_CO2_func(t_grid)



#calculate cumulated amount of CO2 in exhaust gas 
CO2_out_cum = []

for t in t_grid:
    func = V_CO2_func     
    cummulation = scipy.integrate.quad(func,0,t,limit=200)
    
    CO2_out_cum.append(cummulation)
    
df["CO2 L_cum"] = [x[0] for x in CO2_out_cum]

df["CO2 mol_cum"] = (df["CO2 L_cum"] * pressure) / (R * T)


#calculation of recovery rate

Carb_RR= {}
Carb_RR["t"]= t_grid
Carb_RR["given"]=[]
Carb_RR["found"]=[]
Carb_RR["RR"]=[]

for t in t_grid:
    
    #mass balance of Carbon given (Substrate,Biomass,Product)
    nC_Sg = y0[1]/Mw_gluc *6 + df["Glc g_cum"].values[t_grid.index(t)]/Mw_gluc * 6   #initial Substrate + Substrate from Feed
    nC_Xg = y0[0]/Mw_bm
    nC_Pg = y0[2]/Mw_RF *17
    
    amountC_given = nC_Sg + nC_Xg + nC_Pg 
    
    
    
    #mass balance of Carbon found (Substrate,Biomass,Product,CO2)
    nC_Sf = (float(a.dataset["off"]["Glucose [g/L]"].loc[[t]])*V)/Mw_gluc * 6
    nC_Xf = (float(a.dataset["off"]["CDW_calc"].loc[[t]])*V)/ Mw_bm
    nC_Pf = (float(a.dataset["off"]["RF [mg/L]"].loc[[t]])*V /1000)/ Mw_RF * 17
    nC_CO2 = float(df["CO2 mol_cum"].values[t_grid.index(t)])/Mw_CO2

    amountC_found = nC_Sf + nC_Xf + nC_Pf + nC_CO2
    
    RR = (amountC_found / amountC_given) * 100
    
    Carb_RR["given"].append(amountC_given)
    Carb_RR["found"].append(amountC_found)
   
    Carb_RR["RR"].append(RR)
 
df2 = pd.DataFrame.from_dict(Carb_RR)
 

df["RR"] = df2["RR"].values 



C:\Users\walkyre\AppData\Local\Temp/ipykernel_10564/1888174591.py:22: IntegrationWarning:

The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.

C:\Users\walkyre\AppData\Local\Temp/ipykernel_10564/1888174591.py:22: IntegrationWarning:

The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.

C:\Users\walkyre\AppData\Local\Temp/ipykernel_10564/1888174591.py:50: IntegrationWarning:

The maximum number of subdivisions (200) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the i

In [58]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

x = df.index


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter( x= x, y=df["Glc g_cum"],name="glc cum"), secondary_y=True)
fig.add_trace(go.Scatter( x= x, y=df["feedrate pure glucose g/h"]),secondary_y=False)
fig.show()

In [ ]:
def calc_CRR(self,experiment, V, y0):

    Mw_bm = 24.445          #g/mol molecular weight of biomass assuming composition of C=1,H=1.594,N=0.293,O=0.387,P=0.012,S=0.005
    Mw_gluc = 180.156       #g/mol molecular weight of glucose C=6,H=12,O=6
    Mw_RF = 376.36          #g/mol molecular weight of Riboflavin C=17,H=20,N=4,O=6
    F_C_in = c["feedrate_glc"](t_grid)
        
    amountC_given = (y0[0]/Mw_bm)+(y0[1]/Mw_gluc *6)+(y0[2]/Mw_RF *17)

    t_grid = experiment.dataset["off"].index.values
    Carb_RR=[]
    for t in t_grid:
        
        nC_S = float(a.dataset["off"]["Glucose [g/L]"].loc[[t]])*V
        nC_X = float(a.dataset["off"]["CDW_calc"].loc[[t]])*V
        nC_P = float(a.dataset["off"]["RF [mg/L]"].loc[[t]])* V /1000
        nC_CO2_cum = 1
        
        amountC_found = nC_S + nC_X + nC_P+ nC_CO2_cum
        Carb_RR = amountC_found / amountC_given
        
        return Carb_RR